In [1]:
import pickle
import sys

import pandas as pd
from os.path import join
import os
import numpy as np

# # Sk learn




In [14]:
import lightgbm as lgb
from lightgbm import LGBMClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, cohen_kappa_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [45]:
# from imblearn.over_sampling import RandomOverSampler, SVMSMOTE, BorderlineSMOTE, ADASYN

In [4]:
data_folder = "../data/preprocessed/"
X = pd.read_pickle(join(data_folder, "X.pk.zip"))
y = pd.read_pickle(join(data_folder, "Y.pk.zip"))

X_mat = X.values
y_vec = y.values.flatten()
X_train, X_test, y_train, y_test = train_test_split(X_mat, y_vec, test_size=0.12, random_state=123)


In [5]:
# ros = RandomOverSampler(random_state=0)
# X_resampled, y_resampled = SVMSMOTE(sampling_strategy='auto').fit_resample(X_train, y_train)

In [6]:
sys.stdout.write(f"The number of features: {X_mat.shape[1]}\n")
sys.stdout.write(f"TRAIN \
0: {np.sum(y_train == 0)}, 1: {np.sum(y_train == 1)}, 2: {np.sum(y_train == 2)}\n")

# sys.stdout.write(f"Re sample \
# 0: {np.sum(y_resampled == 0)}, 1: {np.sum(y_resampled == 1)}, 2: {np.sum(y_resampled == 2)}\n")

sys.stdout.write(f"TEST  \
0: {np.sum(y_test == 0)}, 1: {np.sum(y_test == 1)}, 2: {np.sum(y_test == 2)}\n")

# test: {X_test.shape[0]}\n")

The number of features: 195
TRAIN 0: 1802, 1: 2487, 2: 57936
TEST  0: 239, 1: 348, 2: 7899


30

# Model

## SKlearn - like

In [8]:
classifiers = {
    # "Tree w": DecisionTreeClassifier(max_depth=8, class_weight='balanced'),
    # "LGBMC": LGBMClassifier(class_weight={0: 5, 1: 5, 2: 1}),
    "LGBMC": LGBMClassifier(),
    "LGBMC bo": LGBMClassifier(
        class_weight='balanced',
    objective = 'multiclass')
}

In [17]:
preprocessor = StandardScaler()
for cla in classifiers:
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor)
        , ('classifier', classifiers[cla])
    ])
    model = pipeline.fit(X_train, y_train)
    predictions = model.predict(X_test)
    score = f1_score(predictions, y_test, average=None)
    score2 = cohen_kappa_score(y_test, predictions)
    print(f'Model:{cla}; score:{score2}; all_score:{score}')

Model:LGBMC; score:0.007278791126420936; all_score:[0.01626016 0.         0.96409818]
Model:LGBMC bo; score:0.03782731570286835; all_score:[0.08225617 0.07469414 0.86010434]


## lgb-like

In [50]:
train_data = lgb.Dataset(X_train, label=y_train)
validation_data = lgb.Dataset(X_test, label=y_test)

In [18]:
def _constant_metric(dy_true, dy_pred):
    """An eval metric that always returns the same value"""
    metric_name = 'constant_metric'
    value = cohen_kappa_score(dy_true, dy_pred)
    is_higher_better = True
    return metric_name, value, is_higher_better

evals_result = {}



In [57]:
y_train

array([1, 2, 2, ..., 2, 2, 2])

In [55]:
#setting up the parameters
params={}
params['learning_rate']=0.03
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='multiclass' #Multi-class target feature
params['metric']='multi_logloss' #metric for multi-class
params['max_depth']=10
params['num_class']=3 
params['num_class']=3 #no.of unique values in the target class not inclusive of the end value

In [75]:
num_round = 800
bst = lgb.train(params, train_data, num_round)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.040891 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18927
[LightGBM] [Info] Number of data points in the train set: 62225, number of used features: 188
[LightGBM] [Info] Start training from score -3.541860
[LightGBM] [Info] Start training from score -3.219680
[LightGBM] [Info] Start training from score -0.071418
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [76]:
ypred = bst.predict(X_test)

In [77]:
ypr = [np.argmax(line) for line in ypred]

In [78]:
cohen_kappa_score(y_test, ypr)

0.010448287767877984

In [25]:
# model = lgb.train(
#     params={
#         # "objective": "multiclass",
#         "metric": "None",
#         # 'num_class' : 3,
#         # "num_iterations": 100,
#         "first_metric_only": True,
#         "verbose": 0,
#         "num_leaves": 8
#     },
#     train_set=train_data,
#     valid_sets=[validation_data],
#     # feval=_constant_metric,
#     early_stopping_rounds=5
# )